# PICOP model of Pelle _et al._ (2019)
<img src="../../doc/Pelle2019/Fig1.png" width="800">

__Figure 1.__ Schematic diagram of PICOP with example data displayed for the Pine Island ice shelf of West Antarctica. The inputs into the parameterization are the basin-averaged ocean temperature ($^\circ$C) and salinity (psu), which are first fed into PICO (red box). PICO uses these inputs to calculate the sub-shelf ambient ocean temperature and salinity fields, which are then used in the plume model (purple box). In addition, the grounding line height is calculated at this time by solving the advection problem defined in the green box. Once these three fields are fed into the plume model, the basal melt rate field is computed according to the steps outlined in the purple box.

In [ ]:
import sys
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt

sys.path.append("..")
matplotlib.rc_file('../rc_file')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

from Plume import PlumeModel
from PICOP import PicoPlumeModel

In [ ]:
da = xr.DataArray(data=np.arange(4).reshape((2,2)), dims=['x','y'], coords={'x':[1,2],'y':[3,4]})

In [ ]:
da.T

In [ ]:
['x', 'y'].pop('x')

In [ ]:
PicoPlumeModel('test2').compute_picop()

## process:

1. geometry
    1. find bounding box
    2. draft of ice shelf
    3. determine grounding line and ice shelf front
2. PICO
    1. determine relative distance
    2. get boxes
    3. set external temperature
3. grounding line depths
    1. select and regrid surface velocity data
    2. solve advection-diffusion equation
4. Plume model
    1. determine dimensionaless coordinate $\hat{X}$
5. 

In [ ]:
def create_plume_dataset(name):
    d1 = xr.open_dataset('../../results/PICO/MoscowUniversity_n2_Ta-0.73_Sa34.73.nc')
    d2 = xr.open_dataset('../../results/PICOP/MoscowUniversity_n2_geometry.nc')

    Ta = xr.zeros_like(d2.draft)
    Ta.name = 'Ta'
    Ta.attrs = {'long_name':'ambient temperature from PICO model', 'units':'degC'}
    Sa = xr.zeros_like(d2.draft)
    Sa.name = 'Sa'
    Sa.attrs = {'long_name':'ambient salinity from PICO model', 'units':'psu'}
    for n in range(1,len(d1.boxnr)):
        Ta += xr.where(d2.box==n, d1.Tk[n], 0.)
        Sa += xr.where(d2.box==n, d1.Sk[n], 0.)
#     alpha = ((d2.alpha.rolling(x=5).mean()+d2.alpha.rolling(y=5).mean())/2)
#     alpha.name = 'alpha'
    dp = xr.merge([d2.dgrl, d2.draft, d2.alpha, d2.grl_adv, Ta, Sa])
    return dp

dp = create_plume_dataset('aa')
ds = PlumeModel(dp).compute_plume()

In [ ]:
d2 = xr.open_dataset('../../results/PICOP/MoscowUniversity_n2_geometry.nc')
alpha = xr.where(ds.alpha, ds.alpha, (d2.mask*ds.alpha.median()))

In [ ]:
d2.mask.plot()

In [ ]:
alpha.plot()

In [ ]:
ds.alpha.plot()

In [ ]:
ds.m.plot(vmin=0, vmax=100, cmap='Reds')

In [ ]:
ds.phi0.plot()

In [ ]:
ds.Phi.plot()